In [1]:
# make sure CUDA is installed
!nvcc --version

# make sure you have a GPU runtime (if this fails go to runtime -> change runtime type)
!nvidia-smi

# Install some magic to run and save .cpp programs
!curl -o ./cpu_runner.py https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/cpu_runner.py
%load_ext cpu_runner

# Install some magic to run and save .cu C++ CUDA programs
!curl -o ./gpu_runner.py https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/gpu_runner.py
%load_ext gpu_runner

# to learn about how to do more fancy things with CUDA using this API see:
# https://nvcc4jupyter.readthedocs.io/en/latest/index.html

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Wed Dec 11 15:43:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8              11W /  7

In [ ]:
%%gpurun -n gpu_knn_example.cu
#include <iostream>
#include <utility>
#include <cmath>
#include <vector>
#include <map>
#include <random>
#include <ctime>
#include <cmath>
#include <chrono>
#include <ctime>
using namespace std;

__device__ float euclidean(int x1, int y1, int x2, int y2){
    float dist = pow((x1 - x2),2) + pow((y1 - y2),2);
    return sqrt(dist);
}

__global__ void knn_searchCUDA(int* d_sample, int* d_neighbor_list, int num_samples, int d, int k) {
    // fill in d_neighbor_list with inf values
    for(int x = 0; x < k*d*num_samples; x++){
        d_neighbor_list[x] = 1000;
    }
    __syncthreads();
    // if thread is 0 or even number --> find neighbors
    // first go through d_sample and find a point that isn't you
    //threadIdx.x == 0 || threadIdx.x % 2 == 0
    if (threadIdx.x == 0 || threadIdx.x % d == 0) {
        int len = num_samples * d;
        for(int i = 0; i < len; i+=d) {
            if (threadIdx.x == i){
                continue;
            }
            printf("neighbor pt (%d, %d) we are looking at\n", d_sample[i], d_sample[i+1]);
            // find dist between you and the other point
            float dist = euclidean(d_sample[threadIdx.x], d_sample[threadIdx.x+1], d_sample[i], d_sample[i+1]);

            // compare the dist with other and insert if necessary
            // find the window of d_neighbor_list you should look through
            int start = (threadIdx.x/2)*k*d; //NOTE: pattern based on 2D space
            int end = start + k*d;
            printf("start: %d, end: %d for thread %d\n", start, end, threadIdx.x);
            int replace_idx = -1;
            float dist_from_curr = dist;
            for(int j = start; j < end; j+=d) {
                float other_dist = euclidean(d_sample[threadIdx.x], d_sample[threadIdx.x+1], d_neighbor_list[j], d_neighbor_list[j+1]);
                // if it's smaller than any of the points, then replace the point furthest away
                printf("other pt (%d, %d) we are looking at\n", d_neighbor_list[j], d_neighbor_list[j+1]);
                if (dist < other_dist) {
                    // check if you've already found a replacement that is farther
                    if (other_dist > dist_from_curr) {
                        dist_from_curr = other_dist;
                        replace_idx = j;
                    }
                }
            }
            if (replace_idx != -1) {
                printf("replacing\n");
                d_neighbor_list[replace_idx] = d_sample[i];
                d_neighbor_list[replace_idx+1] = d_sample[i+1];
            }
        }
    }
    __syncthreads();

}
//assume its a vector representing points
__host__ void print_v(vector<int> v){
    for(int n = 0; n < v.size(); n+=2){
        cout << "(" << v[n] << "," << v[n+1] <<")"<<" ";
    }
    cout << endl;
}
__host__ float c_euclidean(int x1, int y1, int x2, int y2){
    float dist = pow((x1 - x2),2) + pow((y1 - y2),2);
    return sqrt(dist);
}

__host__ float find_path_len(vector<int> v){
    float sum = 0;
    for (int n = 0; n < v.size()-2; n++){ //stop when you reach second to last point
     sum += c_euclidean(v[n], v[n+1], v[n+2], v[n+3]);
    }
    return sqrt(sum);
}

__host__ vector<int> select_min_path(vector<int> a, vector<int> b){
    if (find_path_len(a) < find_path_len(b)) {
        return a;
    }
    return b;
}

__host__ bool is_collision(obstacles, neighbor_x, neighbor_y, tail_x, tail_y) {
    for (int i = 0; i < obstacles.size(); i+=2){
        float slope = (neighbor_y-tail_y)/(neighbor_x-tail_x);
        float b = tail_y - slope*tail_x;
        if (obstacle[i+1] == slope*obstacle[i] + b) {
            return true;
        }
    }
    return false;
}

__host__ void find_shortest_path(int num_samples, int k, int d, int start_x, int start_y, int goal_x, int goal_y, map<vector<int>, vector<int>> knn_map, vector<int> obstacles) {
    // keep track of paths that you haven't expanded on yet
    // each path will be represented by a 1d vector
    vector<vector<int>> unchecked;

    //start as the first path
    vector<int> first_path = {start_x, start_y};
    unchecked.push_back(first_path);

    int count = 0;
    vector<int> min_path = {0,0, 2000, 2000}; //initialize min_path to some infinite length

    while(unchecked.size() != 0) {
        vector<int> curr_path = unchecked[0];
        bool move_on = false;

        // make sure to add new paths to paths along the way
        int len_path = curr_path.size();
        int tail_x = curr_path[len_path-2];
        int tail_y = curr_path[len_path-1];
        vector<int> tail = {tail_x, tail_y};
        printf("tail %d != goal %d || tail %d != %d\n", tail_x, goal_x, tail_y, goal_y);

        // keep building curr path until tail is goal point
        while (tail_x != goal_x || tail_y != goal_y){
            // get the k neighbors of tail
            vector<int> k_neighbors = knn_map[tail];
            printf("Here are the k neighbors: ");
            //printf("first %d\n", k_neighbors[0]);
            print_v(k_neighbors);

            // for the first k-1 neighbors, 1) create new path, 2) connect neighbor to the end, and 3) add to unchecked
            // for last neighbor, 1) connect it to curr_path, 2) continue to building it

            for (int i = 0; i < k*d; i+=d){
                int neighbor_x = k_neighbors[i];
                int neighbor_y = k_neighbors[i+1];
                printf("Looking at neighbor pt (%d, %d): \n", neighbor_x, neighbor_y);
                // check if neighbor already exists in curr_path
                bool next_neighbor = false;
                for (int j = 0; j < curr_path.size(); j+=1){
                    if (neighbor_x == curr_path[j] && neighbor_y == curr_path[j+1]){
                        printf("This neighbor pt already exists in path. Move on.\n");
                        next_neighbor = true;
                        break;
                    }
                }
                //check if connection to neighbor is an obstacle
                if (is_collision(obstacles, neighbor_x, neighbor_y, tail_x, tail_y) == true) {
                    printf("collision\n");
                    continue;
                }
                if (next_neighbor == true) {
                    printf("moving to next neighbor\n");
                    continue;
                }

                if (i != k_neighbors.size()-2){
                    // create a new path
                    vector<int> new_path;
                    copy(curr_path.begin(), curr_path.end(), back_inserter(new_path));
                    new_path.push_back(neighbor_x);
                    new_path.push_back(neighbor_y);
                    printf("Just created new path and inserted neighbor to make this new path:");
                    print_v(new_path);
                    if (neighbor_x == goal_x && neighbor_y == goal_y){ //TODO: check path distance and compare to min path and min dist

                        printf("found a finished path\n");
                        min_path = select_min_path(new_path, min_path);

                    } else { //
                        printf("Added new path to unchecked\n");
                        unchecked.push_back(new_path);
                    }

                } else {
                    printf("At the last neighbor and building curr path which is \n");
                    curr_path.push_back(neighbor_x);
                    curr_path.push_back(neighbor_y);
                    print_v(curr_path);
                    //print("")
                    //vector<int> temp_tail = {curr_path[len_path-2], curr_path[len_path-1]};
                    //printf("AT THE END tail %d != goal %d || tail %d != %d\n", temp_tail[0], goal_x, temp_tail[1], goal_y);
                }

            }

            // if tail didn't change then this curr_path is a dead_end
            vector<int> temp_tail = {curr_path[curr_path.size()-2], curr_path[curr_path.size()-1]};
            if (tail == temp_tail) {
                move_on = true;
                vector<vector<int>> temp_unchecked;
                copy(unchecked.begin()+1, unchecked.end(), back_inserter(temp_unchecked));
                unchecked = temp_unchecked;
                printf("tail of path did NOT change\n");
                break;
            } else {
                printf("tail of path did change\n");
                tail = temp_tail;
            }
            //printf("AT THE END tail %d != goal %d || tail %d != %d\n", tail_x, goal_x, tail_y, goal_y);


        }
        if (move_on == false) { //curr_path was good
            // remove curr_path from unchecked
            //TODO: check against current min_path
            vector<vector<int>> temp_unchecked;
            copy(unchecked.begin()+1, unchecked.end(), back_inserter(temp_unchecked));
            unchecked = temp_unchecked;
            min_path = select_min_path(curr_path, min_path);
        } else { //move_on == true means we've already removed it
            printf("this curr path was not good ");
            print_v(curr_path);
            printf("moving on to the next possible path\n");
        }

        count++;
        /*if (count == 4) {
            printf("reached count. ending now\n");
            break;
        }*/
    }
    printf("Completed search\n");
    if (min_path[min_path.size()-2] == 2000 && min_path[min_path.size()-1]==2000){
        printf("A path from start to goal was not found.\n");
    } else {
        printf("The shortest path from start to goal is ");
        print_v(min_path);
    }


}
__host__ bool already_exists(vector<int> a, vector<int> b, int x, int y){
    for(int i = 0; i < a.size(); i++){
        if(a[i] == x && b[i] == y) {
            return true;
        }
    }
    return false;
}
__host__
int main()
{
    //c_time start_time = time(0);
    std::chrono::time_point<std::chrono::system_clock> start, end;
    start = std::chrono::system_clock::now();
    int d = 2;
    int k = 2;
    int start_x = 0;
    int start_y = 0;
    int goal_x = 3;
    int goal_y = 3;
    int grid_rows = 4;
    int grid_cols = 4;
    float percentage = .2; // generate a sample that is 20% of configuration space
    int num_samples = (int)grid_cols*grid_rows*percentage + 1;
    //Generate random sample of grid space
    srand(1);
    vector<int> my_x, my_y;
    my_x.push_back(goal_x);
    my_y.push_back(goal_y);
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<> distrib1(0, grid_rows);
    uniform_int_distribution<> distrib2(0, grid_cols);
    for(int m = 0; m < num_samples; m++) {
        int rand_x = distrib1(gen);
        int rand_y = distrib2(gen);
        while (already_exists(my_x, my_y, rand_x, rand_y)){
            rand_x = distrib1(gen);
            rand_y = distrib2(gen);

        }
        my_x.push_back(rand_x);
        my_y.push_back(rand_y);
    }

    // Initialize host variables
    //num_samples = 4;
    int* h_sample = (int*)malloc(sizeof(int)*d*(num_samples));
    int len = d*num_samples;
    //if 0 or even, add from list of x
    //if odd, add from list of y
    for(int i = 0; i < len; i++){
        int indx;
        if (i == 0 || i % 2 == 0) {
          indx = i/2;
          h_sample[i] = my_x[indx];
        } else {
            indx = (int)floor(i/2);
            h_sample[i] = my_y[indx];
        }

    }
    int* h_neighbor_list = (int*)malloc(sizeof(int)*d*k*num_samples);

    // Initialize device variables
    int* d_sample;
    int* d_neighbor_list;
    cudaMalloc(&d_sample, sizeof(int)*d*num_samples);
    cudaMalloc(&d_neighbor_list, sizeof(int)*d*k*num_samples);
    cudaMemcpy(d_sample, h_sample, sizeof(int)*d*num_samples, cudaMemcpyHostToDevice);

    // Find neighbors of each sample point in parallel
    knn_searchCUDA<<<1, len>>>(d_sample, d_neighbor_list, num_samples, d, k);
    cudaDeviceSynchronize();

    cudaMemcpy(h_neighbor_list, d_neighbor_list, sizeof(int)*d*k*num_samples, cudaMemcpyDeviceToHost);
    for(int i = 0; i < d*k*num_samples; i++){
        cout << "neighbor" << i << ": " << h_neighbor_list[i] << endl;
    }

    //make knn_map from h_neighbor_list
    map<vector<int>, vector<int>> knn_map;
    for(int i = 0; i < len; i+=d) {
        vector<int> point = {h_sample[i], h_sample[i+1]};
        int start = (i/2)*k*d;
        int end = start + k*d;
        printf("KNN MAP: (%d, %d) start: %d, end: %d\n", point[0], point[1], start, end);
        vector<int> neighbors;
        for(int j = start; j < end; j+=d){
            //neighbors.insert(neighbors.end(), h_neighbor_list[j]);
            //neighbors.insert(neighbors.end(), h_neighbor_list[j+1]);
            neighbors.push_back(h_neighbor_list[j]);
            neighbors.push_back(h_neighbor_list[j+1]);
        }
        knn_map[point] = neighbors;
    }
    // add start to knn_map
    vector<int> start_neighbor_list;
    for(int m = 0; m < d*k; m++){
        start_neighbor_list.push_back(1000);
    }

    for(int i = 0; i < len; i+=d) {
        //printf("neighbor pt (%d, %d) we are looking at\n", h_sample[i], h_sample[i+1]);
        // find dist between start and the other point
        float dist = c_euclidean(start_x, start_y, h_sample[i], h_sample[i+1]);

        // compare the dist with other and insert if necessary
        // go through entire neighbor list of start
        int replace_idx = -1;
        float dist_from_curr = dist;
        for(int j = 0; j < d*k; j+=d) {
            float other_dist = c_euclidean(start_x, start_y, start_neighbor_list[j], start_neighbor_list[j+1]);
            // if it's smaller than any of the points, then replace the point furthest away
            //printf("other pt (%d, %d) we are looking at\n", start_neighbor_list[j], start_neighbor_list[j+1]);
            if (dist < other_dist) {
                // check if you've already found a replacement that is farther
                if (other_dist > dist_from_curr) {
                    dist_from_curr = other_dist;
                    replace_idx = j;
                }
            }
        }
        if (replace_idx != -1) {
            //printf("replacing\n");
            start_neighbor_list[replace_idx] = h_sample[i];
            start_neighbor_list[replace_idx+1] = h_sample[i+1];
        }
    }
    vector<int> start_pt = {start_x, start_y};
    knn_map[start_pt] = start_neighbor_list;

    /*for (const auto& pair : knn_map) {
        //printf("point (%d, %d): ", pair.first[0], pair.first[1]);
        vector<int> my_n = pair.second;
        for (int p : my_n){
            //printf("(%d) ", p);
        }
        printf("\n ");
    }   */
    //add obstacles
    vector<int> obstacles = {1,2,1,0,1,2};
    //vector<int> obstacles = {};

    find_shortest_path(num_samples, k, d, start_x, start_y, goal_x, goal_y, knn_map, obstacles);

    //TODO: free memory
    //c_time end_time = time(0);
    end = std::chrono::system_clock::now();
    std::chrono::duration<double> elapsed_seconds = end - start;
    //printf("%ld", end-start);
    cout << elapsed_seconds.count() << " duration" << endl;
    return 0;
}


In [22]:
%%cpurun -n cpu_knn_example.cpp
#include <iostream>
#include <utility>
#include <cmath>
#include <vector>
#include <map>
#include <random>
#include <cmath>
#include <chrono>
#include <ctime>
using namespace std;

float euclidean(int x1, int y1, int x2, int y2){
    float dist = pow((x1 - x2),2) + pow((y1 - y2),2);
    return sqrt(dist);
}

void knn_search(int* d_sample, int* d_neighbor_list, int num_samples, int d, int k) {
    // fill in d_neighbor_list with inf values
    for(int x = 0; x < k*d*num_samples; x++){
        d_neighbor_list[x] = 1000;
    }
    // for each sample point, go through d_sample and find a point that isn't you
    for (int thread = 0; thread < num_samples*d; thread+=d){
        int len = num_samples * d;
        for(int i = 0; i < len; i+=d) {
            if (thread == i){
                continue;
            }
            printf("neighbor pt (%d, %d) we are looking at\n", d_sample[i], d_sample[i+1]);
            // find dist between you and the other point
            float dist = euclidean(d_sample[thread], d_sample[thread+1], d_sample[i], d_sample[i+1]);

            // compare the dist with other and insert if necessary
            // find the window of d_neighbor_list you should look through
            int start = (thread/2)*k*d; //NOTE: pattern based on 2D space
            int end = start + k*d;
            printf("start: %d, end: %d for thread %d\n", start, end, thread);
            int replace_idx = -1;
            float dist_from_curr = dist;
            for(int j = start; j < end; j+=d) {
                float other_dist = euclidean(d_sample[thread], d_sample[thread+1], d_neighbor_list[j], d_neighbor_list[j+1]);
                // if it's smaller than any of the points, then replace the point furthest away
                printf("other pt (%d, %d) we are looking at\n", d_neighbor_list[j], d_neighbor_list[j+1]);
                if (dist < other_dist) {
                    // check if you've already found a replacement that is farther
                    if (other_dist > dist_from_curr) {
                        dist_from_curr = other_dist;
                        replace_idx = j;
                    }
                }
            }
            if (replace_idx != -1) {
                printf("replacing\n");
                d_neighbor_list[replace_idx] = d_sample[i];
                d_neighbor_list[replace_idx+1] = d_sample[i+1];
            }
        }

    }
    //return d_neighbor_list;

}
//assume its a vector representing points
void print_v(vector<int> v){
    for(int n = 0; n < v.size(); n+=2){
        cout << "(" << v[n] << "," << v[n+1] <<")"<<" ";
    }
    cout << endl;
}
float c_euclidean(int x1, int y1, int x2, int y2){
    float dist = pow((x1 - x2),2) + pow((y1 - y2),2);
    return sqrt(dist);
}

float find_path_len(vector<int> v){
    float sum = 0;
    for (int n = 0; n < v.size()-2; n++){ //stop when you reach second to last point
     sum += c_euclidean(v[n], v[n+1], v[n+2], v[n+3]);
    }
    return sqrt(sum);
}

vector<int> select_min_path(vector<int> a, vector<int> b){
    if (find_path_len(a) < find_path_len(b)) {
        return a;
    }
    return b;
}

bool is_collision(vector<int> obstacles, int neighbor_x, int neighbor_y, int tail_x, int tail_y) {
    for (int i = 0; i < obstacles.size(); i+=2){
        float slope = (neighbor_y-tail_y)/(neighbor_x-tail_x);
        float b = tail_y - slope*tail_x;
        float p = (float)(obstacles[i+1]);
        float q = (float)(slope*obstacles[i] + b);
        printf("%.2f, %.2f\n", p, q);
        if (p == q) {
            return true;
        }
    }
    return false;
}


void find_shortest_path(int num_samples, int k, int d, int start_x, int start_y, int goal_x, int goal_y, map<vector<int>, vector<int>> knn_map, vector<int> obstacles) {
    // keep track of paths that you haven't expanded on yet
    // each path will be represented by a 1d vector
    vector<vector<int>> unchecked;

    //start as the first path
    vector<int> first_path = {start_x, start_y};
    unchecked.push_back(first_path);

    int count = 0;
    vector<int> min_path = {0,0, 2000, 2000}; //initialize min_path to some infinite length

    while(unchecked.size() != 0) {
        vector<int> curr_path = unchecked[0];
        bool move_on = false;

        // make sure to add new paths to paths along the way
        int len_path = curr_path.size();
        int tail_x = curr_path[len_path-2];
        int tail_y = curr_path[len_path-1];
        vector<int> tail = {tail_x, tail_y};
        printf("tail %d != goal %d || tail %d != %d\n", tail_x, goal_x, tail_y, goal_y);

        // keep building curr path until tail is goal point
        while (tail_x != goal_x || tail_y != goal_y){
            // get the k neighbors of tail
            vector<int> k_neighbors = knn_map[tail];
            printf("Here are the k neighbors: ");
            //printf("first %d\n", k_neighbors[0]);
            print_v(k_neighbors);

            // for the first k-1 neighbors, 1) create new path, 2) connect neighbor to the end, and 3) add to unchecked
            // for last neighbor, 1) connect it to curr_path, 2) continue to building it

            for (int i = 0; i < k*d; i+=d){
                int neighbor_x = k_neighbors[i];
                int neighbor_y = k_neighbors[i+1];
                printf("Looking at neighbor pt (%d, %d): \n", neighbor_x, neighbor_y);
                // check if neighbor already exists in curr_path
                bool next_neighbor = false;
                for (int j = 0; j < curr_path.size(); j+=1){
                    if (neighbor_x == curr_path[j] && neighbor_y == curr_path[j+1]){
                        printf("This neighbor pt already exists in path. Move on.\n");
                        next_neighbor = true;
                        break;
                    }
                }
                //check if connection to neighbor is an obstacle
                if (is_collision(obstacles, neighbor_x, neighbor_y, tail_x, tail_y) == true) {
                    printf("collision\n");
                    continue;
                }
                if (next_neighbor == true) {
                    printf("moving to next neighbor\n");
                    continue;
                }

                if (i != k_neighbors.size()-2){
                    // create a new path
                    vector<int> new_path;
                    copy(curr_path.begin(), curr_path.end(), back_inserter(new_path));
                    new_path.push_back(neighbor_x);
                    new_path.push_back(neighbor_y);
                    printf("Just created new path and inserted neighbor to make this new path:");
                    print_v(new_path);
                    if (neighbor_x == goal_x && neighbor_y == goal_y){ //TODO: check path distance and compare to min path and min dist

                        printf("found a finished path\n");
                        min_path = select_min_path(new_path, min_path);

                    } else { //
                        printf("Added new path to unchecked\n");
                        unchecked.push_back(new_path);
                    }

                } else {
                    printf("At the last neighbor and building curr path which is \n");
                    curr_path.push_back(neighbor_x);
                    curr_path.push_back(neighbor_y);
                    print_v(curr_path);
                    //print("")
                    //vector<int> temp_tail = {curr_path[len_path-2], curr_path[len_path-1]};
                    //printf("AT THE END tail %d != goal %d || tail %d != %d\n", temp_tail[0], goal_x, temp_tail[1], goal_y);
                }

            }

            // if tail didn't change then this curr_path is a dead_end
            vector<int> temp_tail = {curr_path[curr_path.size()-2], curr_path[curr_path.size()-1]};
            if (tail == temp_tail) {
                move_on = true;
                vector<vector<int>> temp_unchecked;
                copy(unchecked.begin()+1, unchecked.end(), back_inserter(temp_unchecked));
                unchecked = temp_unchecked;
                printf("tail of path did NOT change\n");
                break;
            } else {
                printf("tail of path did change\n");
                tail = temp_tail;
            }
            //printf("AT THE END tail %d != goal %d || tail %d != %d\n", tail_x, goal_x, tail_y, goal_y);


        }
        if (move_on == false) { //curr_path was good
            // remove curr_path from unchecked
            //TODO: check against current min_path
            vector<vector<int>> temp_unchecked;
            copy(unchecked.begin()+1, unchecked.end(), back_inserter(temp_unchecked));
            unchecked = temp_unchecked;
            min_path = select_min_path(curr_path, min_path);
        } else { //move_on == true means we've already removed it
            printf("this curr path was not good ");
            print_v(curr_path);
            printf("moving on to the next possible path\n");
        }

        count++;
        /*if (count == 4) {
            printf("reached count. ending now\n");
            break;
        }*/
    }
    printf("Completed search\n");
    if (min_path[min_path.size()-2] == 2000 && min_path[min_path.size()-1]==2000){
        printf("A path from start to goal was not found.\n");
    } else {
        printf("The shortest path from start to goal is ");
        print_v(min_path);
    }


}
bool already_exists(vector<int> a, vector<int> b, int x, int y){
    for(int i = 0; i < a.size(); i++){
        if(a[i] == x && b[i] == y) {
            return true;
        }
    }
    return false;
}

int main()
{
    //c_time start_time = time(0);
    std::chrono::time_point<std::chrono::system_clock> start, end;
    start = std::chrono::system_clock::now();
    int d = 2;
    int k = 2;
    int start_x = 0;
    int start_y = 0;
    int goal_x = 30;
    int goal_y = 30;
    int grid_rows = 60;
    int grid_cols = 60;
    float percentage = .2; // generate a sample that is 20% of configuration space
    int num_samples = (int)grid_cols*grid_rows*percentage + 1;
    //Generate random sample of grid space
    vector<int> my_x, my_y;
    my_x.push_back(goal_x);
    my_y.push_back(goal_y);
    srand(1);
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<> distrib1(0, grid_rows);
    uniform_int_distribution<> distrib2(0, grid_cols);
    for(int m = 0; m < num_samples; m++) {
        int rand_x = distrib1(gen);
        int rand_y = distrib2(gen);
        while (already_exists(my_x, my_y, rand_x, rand_y)){
            rand_x = distrib1(gen);
            rand_y = distrib2(gen);

        }
        my_x.push_back(rand_x);
        my_y.push_back(rand_y);
    }

    // Initialize host variables
    //num_samples = 4;
    int* h_sample = (int*)malloc(sizeof(int)*d*(num_samples));
    int len = d*num_samples;
    //if 0 or even, add from list of x
    //if odd, add from list of y
    for(int i = 0; i < len; i++){
        int indx;
        if (i == 0 || i % 2 == 0) {
          indx = i/2;
          h_sample[i] = my_x[indx];
        } else {
            indx = (int)floor(i/2);
            h_sample[i] = my_y[indx];
        }

    }
    int* h_neighbor_list = (int*)malloc(sizeof(int)*d*k*num_samples);

    // Initialize device variables
    int* d_sample;
    int* d_neighbor_list;

    // Find neighbors of each sample point in parallel
    knn_search(h_sample, h_neighbor_list, num_samples, d, k);
    for(int i = 0; i < d*k*num_samples; i++){
        cout << "neighbor" << i << ": " << h_neighbor_list[i] << endl;
    }

    //make knn_map from h_neighbor_list
    map<vector<int>, vector<int>> knn_map;
    for(int i = 0; i < len; i+=d) {
        vector<int> point = {h_sample[i], h_sample[i+1]};
        int start = (i/2)*k*d;
        int end = start + k*d;
        printf("KNN MAP: (%d, %d) start: %d, end: %d\n", point[0], point[1], start, end);
        vector<int> neighbors;
        for(int j = start; j < end; j+=d){
            //neighbors.insert(neighbors.end(), h_neighbor_list[j]);
            //neighbors.insert(neighbors.end(), h_neighbor_list[j+1]);
            neighbors.push_back(h_neighbor_list[j]);
            neighbors.push_back(h_neighbor_list[j+1]);
        }
        knn_map[point] = neighbors;
    }
    // add start to knn_map
    vector<int> start_neighbor_list;
    for(int m = 0; m < d*k; m++){
        start_neighbor_list.push_back(1000);
    }

    for(int i = 0; i < len; i+=d) {
        //printf("neighbor pt (%d, %d) we are looking at\n", h_sample[i], h_sample[i+1]);
        // find dist between start and the other point
        float dist = c_euclidean(start_x, start_y, h_sample[i], h_sample[i+1]);

        // compare the dist with other and insert if necessary
        // go through entire neighbor list of start
        int replace_idx = -1;
        float dist_from_curr = dist;
        for(int j = 0; j < d*k; j+=d) {
            float other_dist = c_euclidean(start_x, start_y, start_neighbor_list[j], start_neighbor_list[j+1]);
            // if it's smaller than any of the points, then replace the point furthest away
            //printf("other pt (%d, %d) we are looking at\n", start_neighbor_list[j], start_neighbor_list[j+1]);
            if (dist < other_dist) {
                // check if you've already found a replacement that is farther
                if (other_dist > dist_from_curr) {
                    dist_from_curr = other_dist;
                    replace_idx = j;
                }
            }
        }
        if (replace_idx != -1) {
            //printf("replacing\n");
            start_neighbor_list[replace_idx] = h_sample[i];
            start_neighbor_list[replace_idx+1] = h_sample[i+1];
        }
    }
    vector<int> start_pt = {start_x, start_y};
    knn_map[start_pt] = start_neighbor_list;

    /*for (const auto& pair : knn_map) {
        //printf("point (%d, %d): ", pair.first[0], pair.first[1]);
        vector<int> my_n = pair.second;
        for (int p : my_n){
            //printf("(%d) ", p);
        }
        printf("\n ");
    }   */
    //add obstacles
    vector<int> obstacles = {1,2,1,0,1,2};
    //vector<int> obstacles = {};

    find_shortest_path(num_samples, k, d, start_x, start_y, goal_x, goal_y, knn_map, obstacles);
    //find_shortest_path(num_samples, k, d, start_x, start_y, goal_x, goal_y, knn_map);

    //TODO: free memory
    //c_time end_time = time(0);
    end = std::chrono::system_clock::now();
    std::chrono::duration<double> elapsed_seconds = end - start;
    //printf("%ld", end-start);
    cout << elapsed_seconds.count() << " duration" << endl;
    return 0;
}

Streaming output truncated to the last 5000 lines.
start: 2880, end: 2884 for thread 1440
other pt (30, 18) we are looking at
other pt (31, 21) we are looking at
neighbor pt (50, 33) we are looking at
start: 2880, end: 2884 for thread 1440
other pt (30, 18) we are looking at
other pt (31, 21) we are looking at
neighbor pt (1, 2) we are looking at
start: 2880, end: 2884 for thread 1440
other pt (30, 18) we are looking at
other pt (31, 21) we are looking at
neighbor pt (60, 7) we are looking at
start: 2880, end: 2884 for thread 1440
other pt (30, 18) we are looking at
other pt (31, 21) we are looking at
neighbor pt (30, 52) we are looking at
start: 2880, end: 2884 for thread 1440
other pt (30, 18) we are looking at
other pt (31, 21) we are looking at
neighbor pt (56, 17) we are looking at
start: 2880, end: 2884 for thread 1440
other pt (30, 18) we are looking at
other pt (31, 21) we are looking at
neighbor pt (55, 53) we are looking at
start: 2880, end: 2884 for thread 1440
other pt (30,